# Overview

References:
* [Building a neural network from scratch](https://www.youtube.com/watch?v=w8yWXqWQYmU)
* [Understanding the math behind neural networks by building one from scratch (no TF/Keras, just numpy)](https://www.samsonzhang.com/2020/11/24/understanding-the-math-behind-neural-networks-by-building-one-from-scratch-no-tf-keras-just-numpy)
* [MNIST database](https://en.wikipedia.org/wiki/MNIST_database)


I'm going to build a neural network from the ground up using Python and basic libraries, without any ML libraries. I'm going to create a neural network to do handwriting recognition against the [MNIST database](https://en.wikipedia.org/wiki/MNIST_database).

Based on the [MNIST docs](http://yann.lecun.com/exdb/mnist/):
```
Each image in the database is 28x28 pixels with a grayscale value for each pixel, ranging between 0 and 255. Pixels are organized row-wise. Pixel values are 0 to 255. 0 means background (white), 255 means foreground (black). 
```

We're going to have $m$ training images.

Call the examples $E$ (a matrix), we would have a matrix where each row has 784 columns.

$
\begin{array}{@{}c@{}}
\begin{bmatrix}
    0 & 0 & 1 & ... & 0 & 0 & 0 \\
    0 & 0 & 0 & ... & 1 & 0 & 0 \\
    &&& ... &&& \\
    0 & 0 & 0 & ... & 0 & 0 & 0 \\
\end{bmatrix}
\qquad
\begin{array}{@{}l@{}}
    <- X^{(1)} \\
    <- X^{(2)} \\
    \\
    <- X^{(m)} \\
\end{array}
\end{array}
$

We will be transposing this to be $X = E^T$, where there are a fixed 784 rows, and a new column for each example. So, this will be a matrix of size $784 \times m$:

$
\begin{bmatrix}
    | & | & | & | \\
    | & | & | & | \\
    | & | & | & | \\
    X^{(1)} & X^{(2)} & ... & X^{(m)} \\
    | & | & | & | \\
    | & | & | & | \\
    | & | & | & | \\
\end{bmatrix}
$

We're going to create a Neural Network with an input layer (784 nodes), one 10-node hidden layers, and a 10-node output layer representing the digits 0 to 9.


# Forward Propagation

We'll call our input $A^{[0]}$, which is just equal to the tranformed matrix with our example data $X$. Therefore, the dimensions of this matrix will be $784 \times m$.

$
A^{[0]} = X
$

$Z^{[1]}$ is the *unactivated* first layer. We're going to apply a weight and a bias to get it.

$
Z^{[1]} = W^{[1]} A^{[0]} + b^{[1]}
$

The dimensions of the matrices involved in calculating $Z^{[1]}$ are as follows:

$
\begin{array}{@{}ll}
    Z^{[1]} \text{ is } 10 \times m \\
    W^{[1]} \text{ is } 10 \times 784 \\
    A^{[0]} \text{ is } 784 \times m \\
    b^{[1]} \text{ is } 10 \times 1 \\
\end{array}
$

Now we need to apply an activation function. We're going to use ReLU (Rectified Linear Unit). If we didn't apply an activation function, each node in the hidden layer would just be a linear combination of the nodes from the previous layer. If you don't apply an activation function, you're effectively just doing a linear regression, but by using a non-linear activation function you add non-linearity to the transformation, which allows more complexity in the solution space. ReLU allows it to be linear instead of using a sigmoid or something else **[research this more to understand it better]**.

$ReLU( x ) = x \text{ if } x > 0\text{; } 0 \text{ if } x <= 0$

We'll call our ReLU-activated output of our input layer $A^{[1]}$

$A^{[1]} \text{ is } 10 \times m$

$A^{[1]} = g( Z^{[1]} ) = ReLU( Z^{[1]} )$

Our unactivated output layer has the following dimensions:

$
\begin{array}{@{}ll}
    Z^{[2]} \text{ is } 10 \times m \\
    W^{[2]} \text{ is } 10 \times 10 \\
    A^{[1]} \text{ is } 10 \times m \\
    b^{[2]} \text{ is } 10 \times 1 \\
\end{array}
$

The unactivated output for the output layer is:

$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$


# Softmax explained

To get the activated output of the layer we apply a ***softmax*** function to get the activated output. Softmax is a function that gives you a percentage normalization by taking each node in the output layer and dividing it by the sum of all of the nodes in the output layer. 

From [Wikipedia](https://en.wikipedia.org/wiki/Softmax_function) we know the softmax function works like this:

$
\text{softmax}(z)_i = \frac{e^{z_i}}{\sum_{j=1}^{K} e^{z_j}}
$

> ℹ️ Note
>
> Don't confuse any of the above variables such as $z$, $i$, $j$ or $K$ with anything we're referencing in this problem. These are just the generic definition of the function as per the Wikipedia page for softmax. More specifically, in the context of the softmax function and its derivative, $i$ and $j$ are indices over the classes. We compute the softmax function for each class $j$, and when computing the derivative of this function, we need to consider two cases, $i = j$ and $i ≠ j$. Separately, as you'll see below during backpropagation within the context of the activated output layer matrix (e.g., $A^{[2]}$), $i$ is the class index and $j$ is the example index. In that case, we're computing the loss function for each example and each class, and when we differentiate the loss with respect to the activations, and $i$ and $j$ are used as indices in this matrix.


The softmax probability vector for the input vector $z$ is a normalized vector where each entry is the probability of the given input value. 

So for example, if we have the following input vector:
$$z = \begin{bmatrix} 
    2.3 \
    -1.7 \
    5.8 \
    0.9 \
    -3.2 \
    7.1 \
    4.6 \
    3.4 \
    -4.8 \
    6.5 \
\end{bmatrix}$$

Then

$$\text{softmax}(z) = \begin{bmatrix} 
    0.005 \
    0.001 \
    0.184 \
    0.002 \
    0.000 \
    0.801 \
    0.008 \
    0.004 \
    0.000 \
    0.016 \
\end{bmatrix}$$ 



# Output

Now that we have our unactivated output layer matrix $Z^{[2]}$, and we know how softmax works, we can use softmax to get our activated output layer probabability distribution. 

Remember, $Z^{[2]} \text{ is } 10 \times m$ so the dimensions of $A^{[2]} \text{ are also } 10 \times m$.

$A^{[2]} = softmax( Z^{[2]} )$

So for example, if $Z^{[2]}$ had the following for one of its examples:
$$Z^{[2]} = \begin{bmatrix} 
    2.3 & ... \\
    -1.7 & ... \\
    5.8 & ... \\
    0.9 & ... \\
    -3.2 & ... \\
    7.1 & ... \\
    4.6 & ... \\
    3.4 & ... \\
    -4.8 & ... \\
    6.5 & ... \\
\end{bmatrix}$$

Then applying that as the input of the softmax function would result in:
$$A^{[2]} = \text{softmax}(Z^{[2]}) = \begin{bmatrix} 
    0.005 & ... \\
    0.001 & ... \\
    0.184 & ... \\
    0.002 & ... \\
    0.000 & ... \\
    0.801 & ... \\
    0.008 & ... \\
    0.004 & ... \\
    0.000 & ... \\
    0.016 & ... \\
\end{bmatrix}$$

Each $A^{[2]}_{i,j}$ value represents the probability that the $j$ example is of class $i$.

If we want to get a "best guess" answer, then we can select the highest probability answer and return that as the selected answer. To do this, we can use a ***one-hot encoding*** function. 

Let's say we have a prediction vector for a single example as $\hat{y}$ as follows:

$$\hat{y} = \begin{bmatrix} 
    0.005 \
    0.001 \
    0.184 \
    0.002 \
    0.000 \
    0.801 \
    0.008 \
    0.004 \
    0.000 \
    0.016 \
\end{bmatrix}$$ 

Then we can define $y$ as the one-hot encoding of the correct label for the training example. If the label for a training example is 5 (because it has the highest probability as in the example above), then the one-hot encoded vector of $y$ would look like this:

$$y = \begin{bmatrix} 0 \ 0 \ 0 \ 0 \ 0 \ 1 \ 0 \ 0 \ 0 \ 0 \ \end{bmatrix}$$

This tells us that given the image input values for this example, the classifier has selected 5 as the best guess on what digit this image is. 


# Loss and Gradient Decent Explained

Backpropagation is the method of taking the loss of the output and using it to calculate changes that we should make to the weights and biases of the preceding layers to move us closer to a more accurate output. Each example will nudge us a little closer to a better solution. The approach we take is to use calculus to calculate the derivative of each component that contributes to the output and adjust accordingly. This approach is known as ***gradient decent***.

Because we're using softmax, we want to use a ***cross-entropy loss function***. We can calculate the loss using the following function:
$$J(\hat{y}, y) = -\sum_{i=0}^{c} y_i \log(\hat{y}_i)$$

Here, $\hat{y}$ is our prediction vector and $y$ is the one-hot encoding of the correct label from the examples in our training data. Because the one-hot vector is $0$ for all entries except for the one with the correct prediction, the loss will just be the $-\log()$ of the probability associated with the correct prediction. 

So, if
$$\hat{y} = \begin{bmatrix} 0.005 \ 0.001 \ 0.184 \ 0.002 \ 0.000 \ 0.801 \ 0.008 \ 0.004 \ 0.000 \ 0.016 \ \end{bmatrix}$$ 
and
$$y = \begin{bmatrix} 0 \ 0 \ 0 \ 0 \ 0 \ 1 \ 0 \ 0 \ 0 \ 0 \ \end{bmatrix}$$
then 
$$J(\hat{y}, y) = -\log(\hat{y}_i) = -\log(0.801) = 0.09636748392 $$

The closer the probability is to $1$ the closer the loss will be to $0$.

Now that we know what loss is, we need to work backwards through our network to find out what portion of the loss is composed by each of the weights and biases. 

$$
\begin{array}{@{}ll}
W^{[1]} := W^{[1]} - \alpha \frac{\delta J}{\delta W^{[1]}} \\
b^{[1]} := b^{[1]} - \alpha \frac{\delta J}{\delta b^{[1]}} \\
W^{[2]} := W^{[2]} - \alpha \frac{\delta J}{\delta W^{[2]}} \\
b^{[2]} := b^{[2]} - \alpha \frac{\delta J}{\delta b^{[2]}} \\
\end{array}
$$


# Backpropagation - Derivative of Loss

In order to calculate the weights and biases for the previous layers, we have to work our way backwards. We have a loss function that is applied to an activated output layer (softmax) that is applied to an unactivated output layer.

So, how can we calculate the derivative of the loss with respect to the unactivated output layer?

Remember our loss function is:
$$J(\hat{y}, y) = -\sum_{i=1}^{c} y_i \log(\hat{y}_i)$$

Let's focus on a single example vector rather than the whole matrix to calculate the derivative. 

For our case, $\hat{y}$ is our prediction vector $A^{[2]}$ and $y$ is our one-hot encoded vector for a single example, which we can call $Y$. So:
$$
\hat{y} = A^{[2]} \text{, } y = Y 
$$

> ℹ️ Note
>
> In this section we're using $A^{[2]}$ and $Y$ to represent a single example vector and $A^{[2]}_i$ and $Y_i$ to refer to a single entry in that vector.

Now we can plug those in:
$$J(\hat{y}, y) = J( A^{[2]}, Y ) = -\sum_{i=1}^{c} Y_i \log(A^{[2]}_i)$$

Now we want to calculate:
$$\frac{\partial J}{\partial A^{[2]}_i}$$

Let's tackle the derivative inside of the summation, then apply the derivative of the summation. We'll bring the negative into the summation. We know that the derivative of $log(x)$ is $1/x$. Differentiating inside the summation we have:
$$ \frac{\partial (-Y_i \log(A^{[2]}_i))}{\partial A^{[2]}_i} = -\frac{Y_i}{A^{[2]}_i}$$

Next, we know that:
$A^{[2]} = \text{softmax}( Z^{[2]} )$

The derivative of the softmax function depends on whether $i=j$ or $i≠j$. In the $i=j$ case, the derivative simplifies to $\text{softmax}(x_i) * (1 - \text{softmax}(x_i))$ and in the $i≠j$ case, it is $-\text{softmax}(x_i) * \text{softmax}(x_j)$.

In our case, that means the derivative of the softmax is (for the $i=j$ case):
$$\frac{\partial A^{[2]}_i}{\partial Z^{[2]}_i} = \text{softmax}'( Z^{[2]}_i ) = \text{softmax}(Z^{[2]}_i) * (1 - \text{softmax}(Z^{[2]}_i)) = A^{[2]}_i * (1 - A^{[2]}_i)$$

Using the chain rule, we can calculate the loss for $\frac{\partial J}{\partial Z^{[2]}_i}$

$$
\frac{\partial J}{\partial Z^{[2]}_i} = \frac{\partial J}{\partial A^{[2]}_i} * \frac{\partial A^{[2]}_i}{\partial Z^{[2]}_i} = -Y_i (1 - A^{[2]}_i)
$$

Because $Y$ is **one-hot** encoded, this simplifies. When considering the derivative of the softmax cross entropy loss with respect to $Z^{[2]}_j$, two cases arise, one when $i$ equals $j$, and one when $i$ doesn't equal $j$. In the first case, the derivative is:
$$Y_j (1 - A^{[2]}_j) \text{ when } i = j$$

And in the second case, it is:
$$-Y_i A^{[2]}_j$$. However, summing over all $i$, only the terms where $i$ equals $j$ survive in the $Y$ vector due to the nature of one-hot encoding. This gives us the difference $A^{[2]}_j - Y_j$, or in vector form $A^{[2]} - Y$.

$$\frac{\partial J}{\partial Z^{[2]}} = A^{[2]} - Y$$


# Backpropagation - Calculating Weights and Biases

Once you have the derivative of the loss function with respect to the linear outputs (in this case $ \frac{\partial J}{\partial Z^{[2]}} $), the next steps in backpropagation involve calculating the gradients with respect to the weights, biases, and activations of previous layers. Here's the procedure:

1. **Calculate the gradient with respect to the weights $W^{[2]}$ of the current layer:**
   
   Using the chain rule, the gradient of the loss with respect to the weights of the output layer can be calculated as:
   $$ \frac{\partial J}{\partial W^{[2]}} = \frac{\partial J}{\partial Z^{[2]}} \times \frac{\partial Z^{[2]}}{\partial W^{[2]}} $$
   
   Given that $Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$, the gradient becomes:
   $$ \frac{\partial J}{\partial W^{[2]}} = dZ^{[2]} A^{[1]T} $$

2. **Calculate the gradient with respect to the biases $b^{[2]}$ of the current layer:**
   
   This can be calculated as:
   $$ \frac{\partial J}{\partial b^{[2]}} = \frac{\partial J}{\partial Z^{[2]}} \times \frac{\partial Z^{[2]}}{\partial b^{[2]}} $$
   
   Given the way biases work (they get added element-wise to each example in the batch), the gradient with respect to the biases becomes a sum across all examples:
   $$ \frac{\partial J}{\partial b^{[2]}} = \sum_{i} dZ^{[2](i)} $$
   where the sum is over all examples in the batch.

3. **Calculate the gradient with respect to the activations of the previous layer $A^{[1]}$:**

   This can be calculated using:
   $$ \frac{\partial J}{\partial A^{[1]}} = \frac{\partial J}{\partial Z^{[2]}} \times \frac{\partial Z^{[2]}}{\partial A^{[1]}} $$
   
   Given $Z^{[2]} = W^{[2]}A^{[1]} + b^{[2]}$, the gradient becomes:
   $$ \frac{\partial J}{\partial A^{[1]}} = W^{[2]T} dZ^{[2]} $$

4. **Move to the Previous Layer:**
   
   To compute the gradients for parameters in the previous layer (i.e., layer 1 in this case), you'd repeat the process, using the activation function of that layer and its derivative. Let's say layer 1 uses the ReLU activation function, then you would need to compute:
   $$ dZ^{[1]} = \frac{\partial J}{\partial A^{[1]}} * g'^{[1]}(Z^{[1]}) $$
   where $g'^{[1]}(Z^{[1]})$ is the derivative of the ReLU function applied element-wise to $Z^{[1]}$.

5. **Update the Weights and Biases:**

   After calculating all the required gradients, use them to update the weights and biases. This is typically done using an optimization algorithm like Gradient Descent or one of its variants:
   $$ W^{[l]} = W^{[l]} - \alpha \frac{\partial J}{\partial W^{[l]}} $$
   $$ b^{[l]} = b^{[l]} - \alpha \frac{\partial J}{\partial b^{[l]}} $$
   where $\alpha$ is the learning rate and $l$ denotes the layer.

6. **Iterate:**

   The above steps are iteratively performed for multiple epochs or until a certain convergence criterion is met, leading to the model's weights and biases being updated to minimize the loss function.

Note: The actual backpropagation calculations are dependent on the specific architecture and activation functions used in the neural network. This summary provides a general idea and is based on the context provided previously.


In [1]:
# Just using tensorflow to get the mnist data
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt



In [9]:
(train_images, train_labels), _ = tf.keras.datasets.mnist.load_data()
data = pd.DataFrame(train_images.reshape(train_images.shape[0], -1))
data['label'] = train_labels

# Get the column names of the DataFrame
columns = data.columns.tolist()

# Move the 'label' column to the first position
label_index = columns.index('label')
columns = ['label'] + columns[:label_index] + columns[label_index+1:]

# Reorder the DataFrame with the new column order
data = data[columns]

# Display the DataFrame with 'label' as the first column
print(data.head())


   label  0  1  2  3  4  5  6  7  8  ...  774  775  776  777  778  779  780  \
0      5  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
1      0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2      4  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
3      1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
4      9  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   

   781  782  783  
0    0    0    0  
1    0    0    0  
2    0    0    0  
3    0    0    0  
4    0    0    0  

[5 rows x 785 columns]


In [63]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

# X is the data
# Y is the labels

data_dev = data[ 0 : 1000 ].T
Y_dev = data_dev[ 0 ]
X_dev = data_dev[ 1 : n ]
X_dev = X_dev / 255

data_train = data[ 1000 : m ].T
Y_train = data_train[ 0 ]
X_train = data_train[ 1 : n ]
X_train = X_train / 255



In [49]:
# print the shape of the training and dev data
print( "X_train (training data): ", X_train.shape )
print( "X_dev (dev data): ", X_dev.shape )

#print the shape of the label data
print( "Y_train (training labels): ", Y_train.shape )
print( "Y_dev (dev labels): ", Y_dev.shape )

X_train (training data):  (784, 59000)
X_dev (dev data):  (784, 1000)
Y_train (training labels):  (59000,)
Y_dev (dev labels):  (1000,)


In [83]:
def init_params():
    # W1 is 10x784
    W1 = np.random.rand( 10, 784 ) - 0.5
    # b1 is 10x1
    b1 = np.random.rand( 10, 1 ) - 0.5
    # W2 is 10x784
    W2 = np.random.rand( 10, 10 ) - 0.5
    # b1 is 10x1
    b2 = np.random.rand( 10, 1 ) - 0.5
    return W1, b1, W2, b2

# goes through Z element wise and returns a new matrix
# 0 if Z[i] < 0
# Z[i] if Z[i] > 0
def ReLU( Z ):
    return np.maximum( Z, 0 ) 

def softmax( Z ):
    # Z -= np.max( Z )
    A = np.exp( Z ) / sum( np.exp( Z ) )
    return A

def forward_prop( W1, b1, W2, b2, X ):
    # Z1 is 10 x m
    Z1 = W1.dot( X ) + b1
    # A is 10 x m
    A1 = ReLU( Z1 )
    # Z2 is 10 x m
    Z2 = W2.dot( A1 ) + b2
    # A2 is 10 x m
    A2 = softmax( Z2 )
    return Z1, A1, Z2, A2

# Y will be a matrix where each column has the
def one_hot( Y ):
    # Create new matrix of size: m x classes
    # m = examples = Y.size
    # classes = Y.max() + 1 = 10
    one_hot_Y = np.zeros( ( Y.size, Y.max() + 1 ) )

    # row - np.arrange( Y.size ) creates an array (the row)
    # column - Y is the labels
    one_hot_Y[ np.arange( Y.size ), Y ] = 1

    # we want each column to be an example, so we transpose
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU( Z ):
    return Z > 0

def back_prop( Z1, A1, Z2, A2, W2, X, Y ):
    m = Y.size
    # print( "m: ", m )
    one_hot_Y = one_hot( Y )
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot( A1.T )
    # print( "back_prop: dZ2.shape: ", dZ2.shape )
    db2 = 1 / m * np.sum( dZ2 )
    dZ1 = W2.T.dot( dZ2 ) * deriv_ReLU( Z1 )
    dW1 = 1 / m * dZ1.dot( X.T ) 
    db1 = 1 / m * np.sum( dZ1 )
    return dW1, db1, dW2, db2

def update_params( W1, b1, W2, b2, dW1, db1, dW2, db2, alpha ):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [88]:
def get_predictions( A2 ):
    return np.argmax( A2, 0 )

def get_accuracy( predictions, Y ):
    print( predictions, Y )
    return np.sum( predictions == Y ) / Y.size
    
def gradient_descent( X, Y, iterations, alpha ):
    W1, b1, W2, b2 = init_params()
    for i in range( iterations ):
        Z1, A1, Z2, A2 = forward_prop( W1, b1, W2, b2, X )
        dW1, db1, dW2, db2 = back_prop( Z1, A1, Z2, A2, W2, X, Y )
        W1, b1, W2, b2 = update_params( W1, b1, W2, b2, dW1, db1, dW2, db2, alpha )
        if i % 25 == 0:
            print( "Iteration: ", i )
            print( "Accuracy: ", get_accuracy( get_predictions( A2 ), Y ))
    return W1, b2, W2, b2

In [90]:
W1, b1, W2, b2 = gradient_descent( X_train, Y_train, 2000, 0.1 )

Iteration:  0
[0 3 6 ... 0 6 0] [4 9 4 ... 7 4 3]
Accuracy:  0.11174576271186441
Iteration:  25
[5 3 4 ... 2 6 5] [4 9 4 ... 7 4 3]
Accuracy:  0.23808474576271185
Iteration:  50
[5 5 4 ... 2 6 5] [4 9 4 ... 7 4 3]
Accuracy:  0.3560847457627119
Iteration:  75
[4 5 4 ... 2 8 5] [4 9 4 ... 7 4 3]
Accuracy:  0.46408474576271186
Iteration:  100
[4 7 7 ... 7 1 5] [4 9 4 ... 7 4 3]
Accuracy:  0.5748813559322034
Iteration:  125
[4 9 7 ... 7 1 5] [4 9 4 ... 7 4 3]
Accuracy:  0.6458983050847458
Iteration:  150
[4 9 7 ... 7 1 5] [4 9 4 ... 7 4 3]
Accuracy:  0.6897796610169491
Iteration:  175
[4 9 4 ... 7 1 3] [4 9 4 ... 7 4 3]
Accuracy:  0.7197457627118644
Iteration:  200
[4 9 4 ... 7 1 3] [4 9 4 ... 7 4 3]
Accuracy:  0.7416440677966102
Iteration:  225
[4 9 4 ... 7 1 3] [4 9 4 ... 7 4 3]
Accuracy:  0.7596440677966102
Iteration:  250
[4 9 4 ... 7 1 3] [4 9 4 ... 7 4 3]
Accuracy:  0.7743220338983051
Iteration:  275
[4 9 4 ... 7 1 3] [4 9 4 ... 7 4 3]
Accuracy:  0.7854915254237288
Iteration:  300
[4